# Apply Modelling to Generate Alternatives (Cartesian Product Algorithm) to a PyPSA-Sec Model Instance of Germany

In [ ]:
import pypsa
import pandas as pd
import itertools
import pyomo.environ as pe

from prepare_model_and_mga import (make_options, 
                                   extra_functionality,
                                   prepare_costs, 
                                   annuity,
                                   make_slacks_mga_weights)
from summary import add_summary_row

idx = pd.IndexSlice

pypsa.Network.lopf_prepare_solver = pypsa.opf.network_lopf_prepare_solver
pypsa.Network.lopf_solve_wo_build = pypsa.opf.network_lopf_solve

options = make_options()

In [ ]:
es = pypsa.Network()

es.import_from_hdf5("all_flex-central_0_DE.h5")

In [ ]:
costs = prepare_costs(file_name = "pypsa-eur-sec-30/data/costs/costs.csv", number_years=1, usd_to_eur=1/1.2, costs_year=2030)

# Add technologies

In [ ]:
es.add("Generator",
         "DE solar-rooftop",
         bus="DE",
         p_nom_extendable=True,
         carrier="solar",
         p_nom_max=es.generators.loc["DE solar","p_nom_max"],
         capital_cost = costs.at[idx['solar-rooftop', 2030],'fixed'],
         p_max_pu=es.generators_t["p_max_pu"].loc[:, "DE solar"],
         marginal_cost=costs.at[idx['solar', 2030],'VOM'])

es.add("Generator", "DE load shedding",
          bus="DE",
          p_nom_extendable=True,
          marginal_cost=1000.)

# Change parameters

In [ ]:
es.global_constraints.at["co2_limit","constant"] = options["co2_limit"]

es.stores.at["DE gas Store", "marginal_cost"] = 100. # renewable gas import, source: ewi Energy Research & Scenarios gGmbH, “Dena-Leitstudie Integrierte Energiewende,” 2018.

# power grid expansion costs (delta to expansion costs of other technologies)
es.generators.at["DE offwind", "capital_cost"] += \
        annuity(costs.loc[idx["offwind", 2030], "lifetime"],costs.loc[idx["offwind", 2030], "discount rate"]) * 900000 

# only 150000 MW of low-cost utility-scale plants; 50:50 for utility:rooftop PV
es.generators.at["DE solar", "p_nom_max"] = 300000. 

# relax p_nom_max for onshore wind
es.generators.at["DE0 onwind", "p_nom_max"] *= 1.1
es.generators.at["DE1 onwind", "p_nom_max"] *= 1.1
es.generators.at["DE2 onwind", "p_nom_max"] *= 1.1

# reduce heat demand for buildings by factor of 0.65 due too buildings retrofitting. This value is 15 percent below (better) than the extrapolation of the trend in the last 25 years. Ausfelder et al., “Sektorkopplung: Untersuchungen und Überlegungen zur Entwicklung eines integrierten Energiesystems,” acatech, Fraunhofer ISE, Berlin, 2017.
 
es.loads_t.p_set["DE heat"] *= 0.65
es.loads_t.p_set["DE urban heat"] *= 0.65

# Initial optimization run

In [ ]:
es.consistency_check()

In [ ]:
es.loads_t.p_set.sum()*3

In [ ]:
87121109917 /((456+458+162)*1e6)

In [ ]:
skip_pre=True

es.lopf(solver_name=options['solver']['name'], solver_options=options['solver']['options'], skip_pre=skip_pre,
          extra_functionality=extra_functionality)

# Prepare MGA

In [ ]:
mga_n = 8
slacks_mga_weights_df = make_slacks_mga_weights(mga_n = mga_n)

In [ ]:
slacks_mga_weights_df.tail()

In [ ]:
es.apply_mga_structure(mga_n=mga_n, slacks_mga_weights_df = slacks_mga_weights_df)

In [ ]:
summary = pd.concat([slacks_mga_weights_df, 
          pd.DataFrame(columns=es.generators.p_nom_opt.index), 
          pd.DataFrame(columns=es.links.p_nom_opt.index), 
          pd.DataFrame(columns=es.storage_units.p_nom_opt.index), 
          pd.DataFrame(columns=es.stores.e_nom_opt.index), 
          pd.DataFrame(columns="p " + es.generators_t.p.columns), 
          pd.DataFrame(columns="p0 " + es.links_t.p0.columns), 
          pd.DataFrame(columns="p1 " + es.links_t.p1.columns), 
          pd.DataFrame(columns="p0 " + es.storage_units_t.p.columns), 
          pd.DataFrame(columns="p1 " + es.storage_units_t.p.columns), 
          pd.DataFrame(columns="p0 " + es.stores_t.p.columns), 
          pd.DataFrame(columns="p1 " + es.stores_t.p.columns)
          ],axis=1)

# Place cost optimal scenario at the end of the result summary df
summary = add_summary_row(df=summary, row_index=len(slacks_mga_weights_df), es=es)
es.export_to_hdf5(path = "results_complete_mga_iterations/iteration_"+str(len(slacks_mga_weights_df))+".h5")

In [ ]:
hasattr(es.model, "mga_function_expr")

# Start mga iterations

In [ ]:
es.lopf_prepare_solver(solver_name=options['solver']['name'])


### Warning: The next code block will take a long time.

In [ ]:
def solve_mga_iterations(mga_n, slacks_mga_weights_df, options, summary_dataframe):
    if not hasattr(es.model, "mga_function_expr"):
        raise TypeError("MGA structure has not been applied (.apply_mga_structure())")
    failed_iterations_i = [] # Save the iterations for which the solver can not find a solution. List should be empty.
    for i, row in slacks_mga_weights_df.iterrows():
        mga_weights_i = list(row.loc[["m"+str(_) for _ in range(mga_n)]])

        es.model.slack = row.loc["slack"]
        for j, w in enumerate(mga_weights_i):
            es.model.mga_weight[j] = w

        if options["solver"]["name"]=="gurobi_persistent":
            es.opt.set_objective(es.model.objective) # Update the objective of the persistent solver instance

            es.opt.remove_constraint(es.model.cost_budget) # Update the cost budget of the persistent solver instance
            es.opt.add_constraint(es.model.cost_budget) # Update the cost budget of the persistent solver instance

        try:
            es.lopf_solve_wo_build(solver_options=options['solver']['options'])

            es.export_to_hdf5(path = "results_complete_mga_iterations/iteration_"+str(i)+".h5")

            summary_dataframe = add_summary_row(df=summary, row_index=i, es=es) # TODO: make summary subsequent to all mga iterations.
            
        except ValueError:
            failed_iterations_i.append(i)
        
    return failed_iterations_i

es.solve_mga_iterations = solve_mga_iterations
failed_iterations_i = es.solve_mga_iterations(mga_n=mga_n, slacks_mga_weights_df=slacks_mga_weights_df, options=options, summary_dataframe=summary)

In [ ]:
failed_iterations_i

# Save tidy result data

In [ ]:
summary.to_csv("result_summary.csv", 
               mode="x"
              )